In [243]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Merge the match data from one year

In [249]:
match_names = ["atp_matches", "atp_matches_futures", "atp_matches_qual_chall"]
frames = []

for name in match_names:
    frames.append(pd.read_csv(f"{name}_2021.csv"))

df = pd.concat(frames)

### Remove redundant columns

In [245]:
# ## irrelevant columns
# irr_cols = []
# player_cols = ["df", "svpt", "1stIn", "1stWon", "2ndWon", "SvGms", "bpSaved", "bpFaced"]
# general_cols = ["draw_size", "round", "loser_seed", "loser_entry", "loser_rank_points", "winner_rank_points", "winner_seed", "winner_entry", "match_num", "tourney_id", "tourney_level", "best_of"]
# ## add winner/loser cols
# for el in player_cols:
#     irr_cols.append(f"w_{el}")
#     irr_cols.append(f"l_{el}")

# ## add the general cols
# irr_cols.extend(general_cols)

# ## drop the cols
# df = df.drop(columns=irr_cols)